## FINAL DEMO

In [458]:
import pandas as pd
import scipy.stats as st
from functions import *

df_final_demo = pd.read_csv("df_final_demo.csv")

df_final_demo.head()
df_final_demo.dropna(inplace=True)

mapGenre = {'X' : 'U'}
df_final_demo["gendr"] = replace_values_df(df_final_demo["gendr"], mapGenre)

## WEB DATA

In [459]:
df_web_data1 = pd.read_csv("df_final_web_data_pt_1.csv")
df_web_data2 = pd.read_csv("df_final_web_data_pt_2.csv")

df_web_data_concat = pd.concat([df_web_data1, df_web_data2], axis=0, join='inner')

In [ ]:
# Convert type object into datetime 

df_web_data_concat['date_time'] = convert_to_dateTime(df_web_data_concat['date_time'])

df_web_data_concat.dtypes

In [461]:
# I turn 'step' into a discrete numeric df

map_values = {'start' : 0, 'step_1' : 1, 'step_2' : 2, 'step_3' : 3, 'confirm' : 4}

df_web_data_concat["process_step"] = replace_values_df(df_web_data_concat["process_step"], map_values)

## FINAL EXP

In [462]:
df_final_exp = pd.read_csv("df_final_experiment_clients.csv")
df_final_exp.rename(columns={"Variation" : "variation"}, inplace=True)

In [463]:
df_final_exp = drop_na_df(df_final_exp, "variation")

In [464]:
map_values2 = {'Control': 0, 'Test': 1}

df_final_exp["variation"] = replace_values_df(df_final_exp["variation"], map_values2)

## MERGE

In [465]:
df_temp1 = pd.merge(df_final_demo, df_web_data_concat, on='client_id', how='inner')

df_all = pd.merge(df_temp1, df_final_exp, on='client_id', how='inner')

In [466]:
df_all = df_all.drop("clnt_tenure_yr", axis=1)      # We don´t need this column because is derivative from clnt_tenure_mnth

In [467]:
df_all_no_duplicates = df_all.copy()
df_all_no_duplicates.drop_duplicates(subset="client_id", inplace=True)

# El dataframe pero sin los duplicados

## QUITAMOS OUTLIERS DEL DF_ALL Y DEL QUE NO TIENE DUPLICADOS

In [468]:
df_all = outlier_slayer(df_all)

df_all_no_duplicates = outlier_slayer(df_all_no_duplicates)

# VER SI SON JOVENES O NO

In [ ]:
df_all_no_duplicates["clnt_age"].describe()

In [ ]:
df_all["clnt_age"].mode()

In [ ]:
import seaborn as sns

sns.boxplot(data = df_all_no_duplicates['clnt_age'])

In [ ]:
Q1 = df_all_no_duplicates['clnt_age'].quantile(0.25)
Q3 = df_all_no_duplicates['clnt_age'].quantile(0.75)
IQR = Q3 - Q1


inf_lim = Q1 - 1.5*IQR
sup_lim = Q3 + 1.5*IQR

inf_lim, sup_lim

In [473]:
# age_outliers = df_all_no_duplicates['clnt_age'][(df_all_no_duplicates['clnt_age'] < inf_lim) | (df_all_no_duplicates['clnt_age'] > sup_lim) ]

# age_outliers         # THERE ARE NO OUTLIERS ON Client Age column

# VER SI SON NUEVOS O ANTIGUOS

In [ ]:
df_all_no_duplicates["clnt_tenure_mnth"].describe()

In [ ]:
df_all["clnt_tenure_mnth"].mode()

In [ ]:
sns.boxplot(data = df_all_no_duplicates['clnt_tenure_mnth'])            # Seguimos teniendo outliers pero dar una segunda pasada es contraproducente en principio.

In [ ]:
Q1 = df_all_no_duplicates['clnt_tenure_mnth'].quantile(0.25)
Q3 = df_all_no_duplicates['clnt_tenure_mnth'].quantile(0.75)
IQR = Q3 - Q1


inf_lim = Q1 - 1.5*IQR
sup_lim = Q3 + 1.5*IQR

inf_lim, sup_lim

# DIA 3  
## Calcular % de compras con éxito en cada web

In [ ]:
df_id_max_compras = df_all.groupby(["client_id", "visit_id", "variation"])["date_time"].idxmax()
df_id_max_compras

In [479]:
df_ambos_portales = df_all.loc[df_id_max_compras]

In [480]:
df_control = df_ambos_portales[df_ambos_portales["variation"] == 0]

In [481]:
df_control_step_0 = df_control[df_control["process_step"] == 0]
df_control_step_1 = df_control[df_control["process_step"] == 1]
df_control_step_2 = df_control[df_control["process_step"] == 2]
df_control_step_3 = df_control[df_control["process_step"] == 3]
df_control_step_4 = df_control[df_control["process_step"] == 4]

In [ ]:
perc_step0_control = 100 * df_control_step_0.shape[0] / df_control.shape[0]
perc_step1_control = 100 * df_control_step_1.shape[0] / df_control.shape[0]
perc_step2_control = 100 * df_control_step_2.shape[0] / df_control.shape[0]
perc_step3_control = 100 * df_control_step_3.shape[0] / df_control.shape[0]

perc_confirms_control = 100 * df_control_step_4.shape[0] / df_control.shape[0]


print(f"""
El % de gente que se quedó en el INICIO en la web de CONTROL es de {perc_step0_control}
El % de gente que se quedó en el PASO 1 en la web de CONTROL es de {perc_step1_control}
El % de gente que se quedó en el PASO 2 en la web de CONTROL es de {perc_step2_control}
El % de gente que se quedó en el PASO 3 en la web de CONTROL es de {perc_step3_control}
El % de gente que CONFIRMÓ la compra en la web de CONTROL es de {perc_confirms_control}
      """)

In [483]:
df_test = df_ambos_portales[df_ambos_portales["variation"] == 1]

In [484]:
df_test_step_0 = df_test[df_test["process_step"] == 0]
df_test_step_1 = df_test[df_test["process_step"] == 1]
df_test_step_2 = df_test[df_test["process_step"] == 2]
df_test_step_3 = df_test[df_test["process_step"] == 3]
df_test_step_4 = df_test[df_test["process_step"] == 4]

In [ ]:
perc_step0_test = 100 * df_test_step_0.shape[0] / df_test.shape[0]
perc_step1_test = 100 * df_test_step_1.shape[0] / df_test.shape[0]
perc_step2_test = 100 * df_test_step_2.shape[0] / df_test.shape[0]
perc_step3_test = 100 * df_test_step_3.shape[0] / df_test.shape[0]

perc_confirms_test = 100 * df_test_step_4.shape[0] / df_test.shape[0]

print(f"""
El % de gente que se quedó en el INICIO en la web de TEST es de {perc_step0_test}
El % de gente que se quedó en el PASO 1 en la web de TEST es de {perc_step1_test}
El % de gente que se quedó en el PASO 2 en la web de TEST es de {perc_step2_test}
El % de gente que se quedó en el PASO 3 en la web de TEST es de {perc_step3_test}
El % de gente que CONFIRMÓ la compra en la web de TEST es de {perc_confirms_test}
      """)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Contar y normalizar las ocurrencias de process_step en cada DataFrame
control_counts = df_control['process_step'].value_counts(normalize=True).sort_index() * 100  
# normalize=True me permite obtener la proporcion. Si no, obtengo datos absolutos y al ser los dos df con distintos número de valores no puedo compararlos 
test_counts = df_test['process_step'].value_counts(normalize=True).sort_index() * 100  

# Crear un DataFrame para facilitar la comparación
counts_df = pd.DataFrame({'Control (%)': control_counts, 'Test (%)': test_counts})

# Graficar
counts_df.plot(kind='bar', figsize=(10, 6))
plt.title('Comparación Proporcional de process_step entre Control y Test')
plt.xlabel('Process Step')
plt.ylabel('Porcentaje de Ocurrencias (%)')
plt.xticks(rotation=0)
plt.legend(title='Grupo')
plt.grid(axis='y')
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Supongamos que df_control y df_test son tus DataFrames con los datos

# Contar y normalizar las ocurrencias de process_step en cada DataFrame
control_counts = df_control['process_step'].value_counts(normalize=True).sort_index() * 100
test_counts = df_test['process_step'].value_counts(normalize=True).sort_index() * 100

# Crear un DataFrame para facilitar la comparación
counts_df = pd.DataFrame({'Control (%)': control_counts, 'Test (%)': test_counts})

# Invertir el orden de los índices para calcular correctamente la probabilidad acumulada desde el paso 0
counts_df = counts_df[::-1]

# Calcular la probabilidad acumulada
counts_df['Control Acumulado (%)'] = counts_df['Control (%)'].cumsum()
counts_df['Test Acumulado (%)'] = counts_df['Test (%)'].cumsum()

# Invertir de nuevo para graficar correctamente
counts_df = counts_df[::-1]

# Graficar
plt.figure(figsize=(10, 6))
plt.plot(counts_df.index, counts_df['Control Acumulado (%)'], marker='o', label='Control Acumulado (%)', color='blue')
plt.plot(counts_df.index, counts_df['Test Acumulado (%)'], marker='o', label='Test Acumulado (%)', color='orange')
plt.title('Probabilidad Acumulada de process_step entre Control y Test')
plt.xlabel('Process Step')
plt.ylabel('Probabilidad Acumulada (%)')
plt.xticks(rotation=0)
plt.legend(title='Grupo')
plt.grid(axis='y')
plt.show()


In [ ]:
df_test

# DIA 4 -> TEST DE HIPÓTESIS

In [489]:
# H0: df_control == df_test
# H1: df_control != df_test

In [ ]:
from statsmodels.stats.proportion import proportions_ztest

# Supongamos que df_control y df_test son tus DataFrames
# Contar el número de clientes que completaron la compra (step 4)
control_complete = df_control[df_control["process_step"] == 4].shape[0]
test_complete = df_test[df_test["process_step"] == 4].shape[0]

# Contar el número total de clientes en cada grupo
control_total = df_control.shape[0]
test_total = df_test.shape[0]

# Calcular las tasas de completación
control_rate = control_complete / control_total
test_rate = test_complete / test_total

# Imprimir las tasas de completación
print(f"Tasa de completación del grupo de control: {control_rate:.2%}")
print(f"Tasa de completación del grupo de test: {test_rate:.2%}")

# Realizar el test de dos proporciones (Z-test)
count = [control_complete, test_complete]
nobs = [control_total, test_total]
stat, p_value = proportions_ztest(count, nobs, alternative='smaller')

# Imprimir los resultados del test
print(f"Estadístico Z: {stat:.4f}")
print(f"P-valor: {p_value:.4f}")

# Interpretar el resultado
alpha = 0.05
if p_value < alpha:
    print("La prueba de control es inferior a la de test")
else:
    print("La diferencia en la tasa de completación no es estadísticamente significativa (no podemos rechazar H0).")


In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency

# Contar el número de clientes que completaron la compra (step 4)
control_complete = df_control[df_control["process_step"] == 4].shape[0]
test_complete = df_test[df_test["process_step"] == 4].shape[0]

# Contar el número de clientes que no completaron la compra
control_incomplete = df_control.shape[0] - control_complete
test_incomplete = df_test.shape[0] - test_complete

# Crear la tabla de contingencia
contingency_table = pd.DataFrame({
    'Complete': [control_complete, test_complete],
    'Incomplete': [control_incomplete, test_incomplete]
}, index=['Control', 'Test'])

# Aplicar el test chi-cuadrado
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Imprimir los resultados del test
print(f"Chi-cuadrado: {chi2:.4f}")
print(f"P-valor: {p:.4f}")

# Interpretar el resultado
alpha = 0.05
if p < alpha:
    print("La diferencia en la tasa de completación es estadísticamente significativa (rechazamos H0).")
else:
    print("La diferencia en la tasa de completación no es estadísticamente significativa (no podemos rechazar H0).")


In [ ]:
# Contar el número de clientes que completaron la compra (step 4)
control_complete = df_control[df_control["process_step"] == 4].shape[0]
test_complete = df_test[df_test["process_step"] == 4].shape[0]

# Contar el número de clientes en cada grupo
n_control = df_control.shape[0]
n_test = df_test.shape[0]

# Calcular la tasa de completación
control_completion_rate = control_complete / n_control
test_completion_rate = test_complete / n_test

print(f"Tasa de completación del grupo Control: {control_completion_rate:.4f}")
print(f"Tasa de completación del grupo Test: {test_completion_rate:.4f}")


In [ ]:
# Calcular el aumento relativo
increase_observed = (test_completion_rate - control_completion_rate) / control_completion_rate * 100

print(f"Aumento observado en la tasa de completación: {increase_observed:.2f}%")


In [ ]:
threshold = 5  # 5%

if increase_observed >= threshold:
    print("El aumento observado cumple con el umbral del 5%.")
else:
    print("El aumento observado no cumple con el umbral del 5%.")


In [495]:
crosstab_result_gen_var = pd.crosstab(df_ambos_portales["gendr"], df_ambos_portales["variation"])

In [ ]:
crosstab_result_gen_var

crosstab_result_gen_var.plot(kind="bar", stacked=True)

In [ ]:
# H0 = There is no association between 'variation' and 'gendr'  --- Null Hypothesis p>=0.05
# H1 = There is an association between 'variation' and 'gendr'  --- Alternative Hypothesis p<0.05

from scipy.stats import chi2_contingency

chi2_statistic, chi2_p_value, _, _ = chi2_contingency(crosstab_result_gen_var)

chi2_statistic, chi2_p_value

if chi2_p_value < 0.05:
    print("We reject the null hypothesis. There is an association between 'variation' and 'gendr'.")
else:
    print("There are no enough evidence that there is an association between 'variation' and 'gendr'.")



In [498]:
crosstab_result_gen_bal = pd.crosstab(df_ambos_portales["gendr"], df_ambos_portales["bal"])

In [ ]:
# H0 = There is no association between 'balance' and 'gendr'  --- Null Hypothesis p>=0.05
# H1 = There is an association between 'balance' and 'gendr'  --- Alternative Hypothesis p<0.05

from scipy.stats import chi2_contingency

chi2_statistic, chi2_p_value, _, _ = chi2_contingency(crosstab_result_gen_bal)

chi2_statistic, chi2_p_value

if chi2_p_value < 0.05:
    print("We reject the null hypothesis. There is an association between 'balance' and 'gendr'.")
else:
    print("There are no enough evidence that there is an association between 'balance' and 'gendr'.")

In [ ]:
# Are Male balance mean greater than Female or Unknowns balance mean?

df_female = df_ambos_portales[(df_ambos_portales["gendr"] == "F")]["bal"]
df_male = df_ambos_portales[df_ambos_portales["gendr"] == "M"]["bal"]
df_unknown = df_ambos_portales[df_ambos_portales["gendr"] == "U"]["bal"]

p_value_MvF = st.ttest_ind(df_male, df_female, alternative="greater")
p_value_FvU = st.ttest_ind(df_female, df_unknown, alternative="greater")

if p_value_MvF[1] < 0.05 and p_value_FvU[1] < 0.05:
    print("We reject the null hypothesis from both hypotheses. Males balance mean are greater than females or unknowns balance mean.")
else:
    print("There are no enough evidence to refuse the null hypothesis.")

In [501]:
# IS THE AVERAGE BALANCE DIFFERENT DEPPENDING ON AGE?

crosstab_result_age_bal = pd.crosstab(df_ambos_portales["clnt_age"], df_ambos_portales["bal"])


In [ ]:
# H0 = There is no association between 'clnt_age' and 'bal'  --- Null Hypothesis p>=0.05
# H1 = There is an association between 'clnt_age' and 'bal'  --- Alternative Hypothesis p<0.05

from scipy.stats import chi2_contingency

chi2_statistic, chi2_p_value, _, _ = chi2_contingency(crosstab_result_gen_bal)

chi2_statistic, chi2_p_value

if chi2_p_value < 0.05:
    print("We reject the null hypothesis. There is an association between 'clnt_age' and 'gendr'.")
else:
    print("There are no enough evidence that there is an association between 'clnt_age' and 'gendr'.")

# DIAS 3 Y 4 DANI

In [ ]:
# Filtrar las compras confirmadas (process_step = 4)
df_all['num_compras'] = df_all['process_step'].apply(lambda x: 1 if x == 4 else 0)          # Creamos columna número de compras

# Calcular ICU
df_all['ICU'] = (df_all['logons_6_mnth'] + df_all['calls_6_mnth'] + df_all['num_compras']) / (df_all['clnt_tenure_mnth'] + df_all['num_accts'])

# Comparar el rendimiento entre portales (1 para nuevo, 0 para antiguo)
kpi_portal_antiguo = df_all[df_all['variation'] == 0]['ICU'].mean()
kpi_portal_nuevo = df_all[df_all['variation'] == 1]['ICU'].mean()

print("ICU - Portal Antiguo:", kpi_portal_antiguo)
print("ICU - Portal Nuevo:", kpi_portal_nuevo)

In [ ]:
# Calcular total de compras y visitas para cada portal
total_visitas_antiguo = df_all[df_all['variation'] == 0]['visit_id'].nunique()
total_compras_antiguo = df_all[(df_all['variation'] == 0) & (df_all['process_step'] == 4)]['visit_id'].nunique()

total_visitas_nuevo = df_all[df_all['variation'] == 1]['visit_id'].nunique()
total_compras_nuevo = df_all[(df_all['variation'] == 1) & (df_all['process_step'] == 4)]['visit_id'].nunique()

tcc_antiguo = total_compras_antiguo / total_visitas_antiguo
tcc_nuevo = total_compras_nuevo / total_visitas_nuevo

print("TCC - Portal Antiguo:", tcc_antiguo)
print("TCC - Portal Nuevo:", tcc_nuevo)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Crea un arreglo con las etiquetas del eje x
x = np.arange(2)

# Crea un gráfico de barras
plt.bar(x - 0.2, [total_compras_antiguo, total_compras_nuevo], label='Compras', width=0.4)
plt.bar(x + 0.2, [total_visitas_antiguo, total_visitas_nuevo], label='Visitas', width=0.4)

# Agrega un título y etiquetas a los ejes
plt.title('Total de Compras y Visitas')
plt.xlabel('Período')
plt.ylabel('Cantidad')

# Agrega etiquetas al eje x
plt.xticks(x, ['Antiguo', 'Nuevo'])

# Agrega una leyenda
plt.legend()

# Muestra el gráfico
plt.show()

In [ ]:
total_compras_antiguo, total_compras_nuevo,total_visitas_antiguo,total_visitas_nuevo

In [ ]:

# Calcular duración de las visitas
df_all['visit_duration'] = df_all.groupby('visit_id')['date_time'].transform(lambda x: (x.max() - x.min()).total_seconds())

# Filtrar las visitas con compras
compras_antiguo = df_all[(df_all['variation'] == 0) & (df_all['process_step'] == 4)]
compras_nuevo = df_all[(df_all['variation'] == 1) & (df_all['process_step'] == 4)]

dpc_antiguo = compras_antiguo['visit_duration'].mean()
dpc_nuevo = compras_nuevo['visit_duration'].mean()

print("DPC - Portal Antiguo:", dpc_antiguo)
print("DPC - Portal Nuevo:", dpc_nuevo)

In [ ]:
df_all.head()

In [ ]:
# Asegurarnos de que 'date_time' esté en formato datetime
df_all['date_time'] = pd.to_datetime(df_all['date_time'])

# Extraer solo la parte de la hora (HH:MM:SS) si es necesario
df_all['time'] = df_all['date_time'].dt.time

# Ordenar los datos por client_id, visit_id y process_step
df_sorted = df_all.sort_values(by=['client_id', 'visit_id', 'process_step'])

# Calcular el tiempo entre pasos, ignorando el primer paso
df_sorted['time_diff'] = df_sorted.groupby(['client_id', 'visit_id'])['date_time'].diff()

# Filtrar los process_step > 0 para evitar valores incorrectos para el primer paso
df_filtered = df_sorted[df_sorted['process_step'] > 0]

# Filtrar los datos por portales
df_antiguo = df_filtered[df_filtered['variation'] == 0]
df_nuevo = df_filtered[df_filtered['variation'] == 1]

# Calcular el tiempo promedio entre pasos para cada portal
avg_time_diff_antiguo = df_antiguo.groupby('process_step')['time_diff'].mean()
avg_time_diff_nuevo = df_nuevo.groupby('process_step')['time_diff'].mean()

# Crear un DataFrame para las comparaciones
df_avg_time_diff = pd.DataFrame({
    'Control': avg_time_diff_antiguo.dt.total_seconds(),
    'Test': avg_time_diff_nuevo.dt.total_seconds()
})

# Gráfico de barras
df_avg_time_diff.plot(kind='bar', figsize=(10, 6))
plt.title('Tiempo Promedio Entre Pasos para Cada Portal')
plt.xlabel('Process Step')
plt.ylabel('Tiempo Promedio (segundos)')
plt.xticks(rotation=0)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Filtrar los process_step > 0 para evitar valores incorrectos para el primer paso
df_filtered = df_sorted[df_sorted['process_step'] > 0].copy()

# Calcular el tiempo acumulado por visita y por paso
# Primero, calculamos el tiempo del paso anterior (equivalente a lag)
df_filtered['time_diff_shifted'] = df_filtered.groupby(['client_id', 'visit_id'])['time_diff'].shift()

# Luego, sumamos el tiempo acumulado de los pasos anteriores para cada cliente y visita
df_filtered['time_accumulated'] = df_filtered.groupby(['client_id', 'visit_id'])['time_diff_shifted'].cumsum().fillna(pd.Timedelta(seconds=0))

# Filtrar nuevamente por portales
df_control_accum = df_filtered[df_filtered['variation'] == 0]
df_test_accum = df_filtered[df_filtered['variation'] == 1]

# Calcular el tiempo acumulado promedio por cada paso en segundos
acc_time_antiguo = df_control_accum.groupby('process_step')['time_accumulated'].mean().dt.total_seconds()
acc_time_nuevo = df_test_accum.groupby('process_step')['time_accumulated'].mean().dt.total_seconds()

# Crear un DataFrame para las comparaciones
df_acc_time = pd.DataFrame({
    'Portal Antiguo': acc_time_antiguo,
    'Portal Nuevo': acc_time_nuevo
})

# Eliminar filas con valores NaN en cualquier columna (si las hay)
df_acc_time.dropna(inplace=True)

# Crear un DataFrame con el punto (0, 0)
df_zero_point = pd.DataFrame({
    'Portal Antiguo': [0],
    'Portal Nuevo': [0]
}, index=[0])

# Concatenar el punto (0, 0) con los datos acumulados
df_acc_time = pd.concat([df_zero_point, df_acc_time])

# Crear el gráfico de líneas, incluyendo el punto (0, 0)
df_acc_time.plot(kind='line', figsize=(10, 6), marker='o')
plt.title('Tiempo Acumulado Promedio por Proceso para Cada Portal (Incluyendo Punto 0)')
plt.xlabel('Process Step')
plt.ylabel('Tiempo Acumulado (segundos)')
plt.grid(True)
plt.show()


In [ ]:
# Visit duration depends on variation?

control_times = df_control_accum['time_accumulated'].dt.total_seconds()
test_times = df_test_accum['time_accumulated'].dt.total_seconds()

st.ttest_ind(test_times, control_times, alternative="greater")

# We can confimr that the test time is greater than the control time with a 95% confidence

# DIA 5

In [ ]:
df_control

In [513]:
df_ambos_portales['confirmed_purchase'] = df_ambos_portales['process_step'].apply(lambda x: 1 if x == 4 else 0)
crosstab_result_age_confirm = pd.crosstab(df_ambos_portales['clnt_age'], df_ambos_portales['confirmed_purchase'])

In [ ]:
# H0 = There is no association between 'balance' and 'gendr'  --- Null Hypothesis p>=0.05
# H1 = There is an association between 'balance' and 'gendr'  --- Alternative Hypothesis p<0.05

from scipy.stats import chi2_contingency

chi2_statistic, chi2_p_value, _, _ = chi2_contingency(crosstab_result_age_confirm)

chi2_statistic, chi2_p_value

if chi2_p_value < 0.05:
    print("We reject the null hypothesis. There is an association between 'age' and confirming purchase.")
else:
    print("There is no significant evidence to suggest an association between 'age' and confirming purchase.")